## Importing & Reading Data

In [1]:
# Importing library that will be use
import numpy as np # For array manipulation
import pandas as pd # For dataframe manipulation
import requests # For requesting data from API
import json # For reading and formating json file

In [2]:
# Creating dataframe and take a look
card_list = pd.read_csv('yugioh.csv')
card_list

,card_name
0,Tornado Dragon
1,Exodia the Forbidden One
2,Dark Magician
3,Kuriboh
4,Obelisk The Tormentor
5,Pot of Greed
6,Yami
7,Mystical Elf
8,Monster Reborn
9,Slifer the Sky Dragon


## Data Preperation for Enriching the data

In [3]:
# Create a list that contain card name
card_list = card_list.card_name.to_list()
card_list

['Tornado Dragon',
 'Exodia the Forbidden One',
 'Dark Magician',
 'Kuriboh',
 'Obelisk The Tormentor',
 'Pot of Greed',
 'Yami',
 'Mystical Elf',
 'Monster Reborn',
 'Slifer the Sky Dragon']

Testing if data is available in Public API

In [4]:
API = 'https://db.ygoprodeck.com/api/v7/cardinfo.php'

params = {'name': ['Tornado Dragon']}
endpoint_get = API
r = requests.get(endpoint_get, params = params)

In [5]:
# See the result
r.json()

{'data': [{'id': 6983839,
   'name': 'Tornado Dragon',
   'type': 'XYZ Monster',
   'desc': '2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.',
   'atk': 2100,
   'def': 2000,
   'level': 4,
   'race': 'Wyrm',
   'attribute': 'WIND',
   'card_sets': [{'set_name': 'Battles of Legend: Relentless Revenge',
     'set_code': 'BLRR-EN084',
     'set_rarity': 'Secret Rare',
     'set_rarity_code': '(ScR)',
     'set_price': '4.65'},
    {'set_name': 'Duel Devastator',
     'set_code': 'DUDE-EN019',
     'set_rarity': 'Ultra Rare',
     'set_rarity_code': '(UR)',
     'set_price': '1.58'},
    {'set_name': 'Legendary Duelists: Synchro Storm',
     'set_code': 'LED8-EN055',
     'set_rarity': 'Common',
     'set_rarity_code': '(C)',
     'set_price': '0.96'},
    {'set_name': 'Maximum Crisis',
     'set_code': 'MACR-EN081',
     'set_rarity': 'Secret Rare',
     'set_rarity_code': '(ScR)',
     'set_pr

Let's try to loop fetch 10 cards

In [6]:
# Initiate dict to collect card info
str_from_dict=''

# Create the loop
for card in card_list:
    # Set the parameters of wanted card
    params = {'name': card}

    # Requesting the data
    endpoint_get = API
    r = requests.get(endpoint_get, params = params)
    d_card = r.json()

    # Creating a list of the data
    if str_from_dict == '':
        str_from_dict += '['
        str_from_dict+=json.dumps(d_card)  
    else:
        str_from_dict+=','+json.dumps(d_card)
        
# Var last stores every card JSON, in string format
last = str_from_dict + ']'

# Reload card info as JSON
data = json.loads(last)

In [7]:
#Check the data
data

[{'data': [{'id': 6983839,
    'name': 'Tornado Dragon',
    'type': 'XYZ Monster',
    'desc': '2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.',
    'atk': 2100,
    'def': 2000,
    'level': 4,
    'race': 'Wyrm',
    'attribute': 'WIND',
    'card_sets': [{'set_name': 'Battles of Legend: Relentless Revenge',
      'set_code': 'BLRR-EN084',
      'set_rarity': 'Secret Rare',
      'set_rarity_code': '(ScR)',
      'set_price': '4.65'},
     {'set_name': 'Duel Devastator',
      'set_code': 'DUDE-EN019',
      'set_rarity': 'Ultra Rare',
      'set_rarity_code': '(UR)',
      'set_price': '1.58'},
     {'set_name': 'Legendary Duelists: Synchro Storm',
      'set_code': 'LED8-EN055',
      'set_rarity': 'Common',
      'set_rarity_code': '(C)',
      'set_price': '0.96'},
     {'set_name': 'Maximum Crisis',
      'set_code': 'MACR-EN081',
      'set_rarity': 'Secret Rare',
      'set_rarity_

## Merge the data

Normalize the data and see the data

In [8]:
df = pd.json_normalize(data)
df

,data
0,"[{'id': 6983839, 'name': 'Tornado Dragon', 'ty..."
1,"[{'id': 33396948, 'name': 'Exodia the Forbidde..."
2,"[{'id': 46986421, 'name': 'Dark Magician', 'ty..."
3,"[{'id': 40640057, 'name': 'Kuriboh', 'type': '..."
4,"[{'id': 10000000, 'name': 'Obelisk the Torment..."
5,"[{'id': 55144522, 'name': 'Pot of Greed', 'typ..."
6,"[{'id': 59197169, 'name': 'Yami', 'type': 'Spe..."
7,"[{'id': 15025844, 'name': 'Mystical Elf', 'typ..."
8,"[{'id': 83764719, 'name': 'Monster Reborn', 't..."
9,"[{'id': 10000020, 'name': 'Slifer the Sky Drag..."


The data still not ready beacuse the JSON is nested in **data columns**

In [9]:
df = pd.json_normalize(data, 'data')
df.head(2)

,id,name,type,desc,atk,def,level,race,attribute,card_sets,card_images,card_prices,archetype,banlist_info.ban_tcg,banlist_info.ban_ocg,banlist_info.ban_goat
0,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,[{'set_name': 'Battles of Legend: Relentless R...,"[{'id': 6983839, 'image_url': 'https://images....","[{'cardmarket_price': '0.19', 'tcgplayer_price...",NaN,NaN,NaN,NaN
1,33396948,Exodia the Forbidden One,Effect Monster,"If you have ""Right Leg of the Forbidden One"", ...",1000.0,1000.0,3.0,Spellcaster,DARK,"[{'set_name': 'Dark Beginning 1', 'set_code': ...","[{'id': 33396948, 'image_url': 'https://images...","[{'cardmarket_price': '0.46', 'tcgplayer_price...",Exodia,Limited,Limited,Limited


The data still nested in **card_sets, card_images, and card_prices columns**

In [10]:
# explode all columns with lists of dicts

df = df.explode("card_sets").reset_index().drop("index",1).explode("card_images").reset_index()\
    .drop("index",1).explode("card_prices").reset_index().drop("index",1)

# list of columns with dicts
cols_to_normalize = ['card_sets', 'card_images','card_prices']

# if there are keys, which will become column names, overlap with excising column names
# add the current column name as a prefix
normalized = list()
for col in cols_to_normalize:
    
    d = pd.json_normalize(df[col])
    d.columns = [f'{col}_{v}' for v in d.columns]
    normalized.append(d.copy())

# combine df with the normalized columns
df = pd.concat([df] + normalized, axis=1).drop(columns=cols_to_normalize)

In [14]:
# Displays card image in the df, based on links
from IPython.core.display import display,HTML

def path_to_image_html(path):
    return '<img src="'+ path + '" width="600" >'

pd.set_option('display.max_colwidth', None) #define df column width to fit the image

image_cols = ['card_images_image_url']  #define which columns will be used to convert to html

# Create the dictionary to be passed as formatters
format_dict = {}
for image_col in image_cols:
    format_dict[image_col] = path_to_image_html

#Display DF with loaded card images
display(HTML(df.to_html(escape=False ,formatters=format_dict)))

,id,name,type,desc,atk,def,level,race,attribute,archetype,banlist_info.ban_tcg,banlist_info.ban_ocg,banlist_info.ban_goat,card_sets_set_name,card_sets_set_code,card_sets_set_rarity,card_sets_set_rarity_code,card_sets_set_price,card_images_id,card_images_image_url,card_images_image_url_small,card_prices_cardmarket_price,card_prices_tcgplayer_price,card_prices_ebay_price,card_prices_amazon_price,card_prices_coolstuffinc_price
0,6983839,Tornado Dragon,XYZ Monster,"2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.",2100.0,2000.0,4.0,Wyrm,WIND,NaN,NaN,NaN,NaN,Battles of Legend: Relentless Revenge,BLRR-EN084,Secret Rare,(ScR),4.65,6983839,,https://images.ygoprodeck.com/images/cards_small/6983839.jpg,0.19,0.15,1.90,1.39,0.79
1,6983839,Tornado Dragon,XYZ Monster,"2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.",2100.0,2000.0,4.0,Wyrm,WIND,NaN,NaN,NaN,NaN,Duel Devastator,DUDE-EN019,Ultra Rare,(UR),1.58,6983839,,https://images.ygoprodeck.com/images/cards_small/6983839.jpg,0.19,0.15,1.90,1.39,0.79
2,6983839,Tornado Dragon,XYZ Monster,"2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.",2100.0,2000.0,4.0,Wyrm,WIND,NaN,NaN,NaN,NaN,Legendary Duelists: Synchro Storm,LED8-EN055,Common,(C),0.96,6983839,,https://images.ygoprodeck.com/images/cards_small/6983839.jpg,0.19,0.15,1.90,1.39,0.79
3,6983839,Tornado Dragon,XYZ Monster,"2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.",2100.0,2000.0,4.0,Wyrm,WIND,NaN,NaN,NaN,NaN,Maximum Crisis,MACR-EN081,Secret Rare,(ScR),4.49,6983839,,https://images.ygoprodeck.com/images/cards_small/6983839.jpg,0.19,0.15,1.90,1.39,0.79
4,33396948,Exodia the Forbidden One,Effect Monster,"If you have ""Right Leg of the Forbidden One"", ""Left Leg of the Forbidden One"", ""Right Arm of the Forbidden One"" and ""Left Arm of the Forbidden One"" in addition to this card in your hand, you win the Duel.",1000.0,1000.0,3.0,Spellcaster,DARK,Exodia,Limited,Limited,Limited,Dark Beginning 1,DB1-EN139,Ultra Rare,(UR),40.1,33396948,,https://images.ygoprodeck.com/images/cards_small/33396948.jpg,0.46,0.83,24.98,1.29,29.99
5,33396948,Exodia the Forbidden One,Effect Monster,"If you have ""Right Leg of the Forbidden One"", ""Left Leg of the Forbidden One"", ""Right Arm of the Forbidden One"" and ""Left Arm of the Forbidden One"" in addition to this card in your hand, you win the Duel.",1000.0,1000.0,3.0,Spellcaster,DARK,Exodia,Limited,Limited,Limited,Dark Legends,DLG1-EN022,Ultra Rare,(UR),124.91,33396948,,https://images.ygoprodeck.com/images/cards_small/33396948.jpg,0.46,0.83,24.98,1.29,29.99
6,33396948,Exodia the Forbidden One,Effect Monster,"If you have ""Right Leg of the Forbidden One"", ""Left Leg of the Forbidden One"", ""Right Arm of the Forbidden One"" and ""Left Arm of the Forbidden One"" in addition to this card in your hand, you win the Duel.",1000.0,1000.0,3.0,Spellcaster,DARK,Exodia,Limited,Limited,Limited,Duelist League 2 participation cards,DL11-EN006,Rare,(R),35.39,33396948,,https://images.ygoprodeck.com/images/cards_small/33396948.jpg,0.46,0.83,24.98,1.29,29.99
7,33396948,Exodia the Forbidden One,Effect Monster,"If you have ""Right Leg of the Forbidden One"", ""Left Leg of the Forbidden One"", ""Right Arm of the Forbidden One"" and ""Left Arm of the Forbidden One"" in addition to this card in your hand, you win the Duel.",1000.0,1000.0,3.0,Spellcaster,DARK,Exodia,Limited,Limited,Limited,Legend of Blue Eyes White Dragon,LOB-124,Ultra Rare,(UR),33.07,33396948,,https://images.ygoprodeck.com/images/cards_small/33396948.jpg,0.46,0.83,24.98,1.29,29.99
8,33396948,Exodia the Forbidden One,Effect Monster,"If you have ""Right Leg of the Forbidden One"", ""Left Leg of

In [12]:
# The final dataframe
df.head(5)

,id,name,type,desc,atk,def,level,race,attribute,archetype,...,card_sets_set_rarity_code,card_sets_set_price,card_images_id,card_images_image_url,card_images_image_url_small,card_prices_cardmarket_price,card_prices_tcgplayer_price,card_prices_ebay_price,card_prices_amazon_price,card_prices_coolstuffinc_price
0,6983839,Tornado Dragon,XYZ Monster,"2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.",2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(ScR),4.65,6983839,https://images.ygoprodeck.com/images/cards/6983839.jpg,https://images.ygoprodeck.com/images/cards_small/6983839.jpg,0.19,0.15,1.90,1.39,0.79
1,6983839,Tornado Dragon,XYZ Monster,"2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.",2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(UR),1.58,6983839,https://images.ygoprodeck.com/images/cards/6983839.jpg,https://images.ygoprodeck.com/images/cards_small/6983839.jpg,0.19,0.15,1.90,1.39,0.79
2,6983839,Tornado Dragon,XYZ Monster,"2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.",2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(C),0.96,6983839,https://images.ygoprodeck.com/images/cards/6983839.jpg,https://images.ygoprodeck.com/images/cards_small/6983839.jpg,0.19,0.15,1.90,1.39,0.79
3,6983839,Tornado Dragon,XYZ Monster,"2 Level 4 monsters\nOnce per turn (Quick Effect): You can detach 1 material from this card, then target 1 Spell/Trap on the field; destroy it.",2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(ScR),4.49,6983839,https://images.ygoprodeck.com/images/cards/6983839.jpg,https://images.ygoprodeck.com/images/cards_small/6983839.jpg,0.19,0.15,1.90,1.39,0.79
4,33396948,Exodia the Forbidden One,Effect Monster,"If you have ""Right Leg of the Forbidden One"", ""Left Leg of the Forbidden One"", ""Right Arm of the Forbidden One"" and ""Left Arm of the Forbidden One"" in addition to this card in your hand, you win the Duel.",1000.0,1000.0,3.0,Spellcaster,DARK,Exodia,...,(UR),40.1,33396948,https://images.ygoprodeck.com/images/cards/33396948.jpg,https://images.ygoprodeck.com/images/cards_small/33396948.jpg,0.46,0.83,24.98,1.29,29.99


In [34]:
#export dataframe to csv
df.to_csv('yugioh_enriched.csv', index = False, encoding='utf-8')

In [35]:
#save image from list of url

#convert columns to list
image_links = df['card_images_image_url_small'].to_list()
image_links = list(dict.fromkeys(image_links)) #remove duplicate links


In [36]:
#save to local directory as .zip
import requests
import zipfile
import os

with zipfile.ZipFile('yugioh_images.zip', 'w') as img_zip:
    for link in image_links: 
        img_name = os.path.basename(link) #define filename
        img_data = requests.get(link).content #request images
        img_zip.writestr(img_name, img_data) #write to local

## Exploration

In [41]:
# See the shape of the data
print(f'There is {df.shape[0]} rows in the dataframe')
print(f'There is {df.shape[1]} columns in the dataframe')

There is 795 rows in the dataframe
There is 26 columns in the dataframe


In [44]:
# List of columns
print('List of all columns can be seen below:')
for colum in df.columns:
    print(colum)

List of all columns can be seen below:
id
name
type
desc
atk
def
level
race
attribute
archetype
banlist_info.ban_tcg
banlist_info.ban_ocg
banlist_info.ban_goat
card_sets_set_name
card_sets_set_code
card_sets_set_rarity
card_sets_set_rarity_code
card_sets_set_price
card_images_id
card_images_image_url
card_images_image_url_small
card_prices_cardmarket_price
card_prices_tcgplayer_price
card_prices_ebay_price
card_prices_amazon_price
card_prices_coolstuffinc_price


In [25]:
# See the null value
df.isnull().sum()

id                                  0
name                                0
type                                0
desc                                0
atk                                95
def                                95
level                              95
race                                0
attribute                          95
archetype                          97
banlist_info.ban_tcg              690
banlist_info.ban_ocg              690
banlist_info.ban_goat             690
card_sets_set_name                  0
card_sets_set_code                  0
card_sets_set_rarity                0
card_sets_set_rarity_code           0
card_sets_set_price                 0
card_images_id                      0
card_images_image_url               0
card_images_image_url_small         0
card_prices_cardmarket_price        0
card_prices_tcgplayer_price         0
card_prices_ebay_price              0
card_prices_amazon_price            0
card_prices_coolstuffinc_price      0
dtype: int64

In [37]:
# it can be seen that the atk, def, and, level of a spell card is null
df.loc[df['atk'].isnull(),:].head(5)

,id,name,type,desc,atk,def,level,race,attribute,archetype,...,card_sets_set_rarity_code,card_sets_set_price,card_images_id,card_images_image_url,card_images_image_url_small,card_prices_cardmarket_price,card_prices_tcgplayer_price,card_prices_ebay_price,card_prices_amazon_price,card_prices_coolstuffinc_price
642,55144522,Pot of Greed,Spell Card,Draw 2 cards.,NaN,NaN,NaN,Normal,NaN,Greed,...,(MSR),4.98,55144522,https://images.ygoprodeck.com/images/cards/551...,https://images.ygoprodeck.com/images/cards_sma...,0.99,2.52,3.95,6.18,3.99
643,55144522,Pot of Greed,Spell Card,Draw 2 cards.,NaN,NaN,NaN,Normal,NaN,Greed,...,(R),5.47,55144522,https://images.ygoprodeck.com/images/cards/551...,https://images.ygoprodeck.com/images/cards_sma...,0.99,2.52,3.95,6.18,3.99
644,55144522,Pot of Greed,Spell Card,Draw 2 cards.,NaN,NaN,NaN,Normal,NaN,Greed,...,(R),3.94,55144522,https://images.ygoprodeck.com/images/cards/551...,https://images.ygoprodeck.com/images/cards_sma...,0.99,2.52,3.95,6.18,3.99
645,55144522,Pot of Greed,Spell Card,Draw 2 cards.,NaN,NaN,NaN,Normal,NaN,Greed,...,(SFR),3.54,55144522,https://images.ygoprodeck.com/images/cards/551...,https://images.ygoprodeck.com/images/cards_sma...,0.99,2.52,3.95,6.18,3.99
646,55144522,Pot of Greed,Spell Card,Draw 2 cards.,NaN,NaN,NaN,Normal,NaN,Greed,...,(SR),40.57,55144522,https://images.ygoprodeck.com/images/cards/551...,https://images.ygoprodeck.com/images/cards_sma...,0.99,2.52,3.95,6.18,3.99


In [34]:
# See list of spell card in the data
df.loc[df['atk'].isnull(),:]['name'].value_counts()

Monster Reborn    70
Pot of Greed      18
Yami               7
Name: name, dtype: int64

In [36]:
# This means the card is not ban to be use in tcg competition
df.loc[df['banlist_info.ban_tcg'].isnull(),:].head(5)

,id,name,type,desc,atk,def,level,race,attribute,archetype,...,card_sets_set_rarity_code,card_sets_set_price,card_images_id,card_images_image_url,card_images_image_url_small,card_prices_cardmarket_price,card_prices_tcgplayer_price,card_prices_ebay_price,card_prices_amazon_price,card_prices_coolstuffinc_price
0,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(ScR),4.65,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.25
1,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(UR),1.58,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.25
2,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(C),0.96,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.25
3,6983839,Tornado Dragon,XYZ Monster,2 Level 4 monsters\nOnce per turn (Quick Effec...,2100.0,2000.0,4.0,Wyrm,WIND,NaN,...,(ScR),4.49,6983839,https://images.ygoprodeck.com/images/cards/698...,https://images.ygoprodeck.com/images/cards_sma...,0.19,0.15,1.90,1.39,0.25
21,46986421,Dark Magician,Normal Monster,''The ultimate wizard in terms of attack and d...,2500.0,2100.0,7.0,Spellcaster,DARK,Dark Magician,...,(UR),6.97,46986421,https://images.ygoprodeck.com/images/cards/469...,https://images.ygoprodeck.com/images/cards_sma...,0.02,0.54,10.95,14.45,0.49


In [43]:
# This is the list of set name from the listed card
for x in df.card_sets_set_name.unique():
    print(x)

Battles of Legend: Relentless Revenge
Duel Devastator
Legendary Duelists: Synchro Storm
Maximum Crisis
Dark Beginning 1
Dark Legends
Duelist League 2 participation cards
Legend of Blue Eyes White Dragon
Legendary Collection 3: Yugi's World Mega Pack
Legendary Decks II
Master Collection Volume 1
Maximum Gold: El Dorado
Premium Gold: Return of the Bling
Retro Pack
The Lost Art Promotion D
Yu-Gi-Oh! Dark Duel Stories promotional cards
Yu-Gi-Oh! GX Ultimate Beginner's Pack
Yugi's Legendary Decks
2016 Mega-Tins
2017 Mega-Tins
Booster Pack Collectors Tins 2002
Collectible Tins 2003
Duel Power
Duel Terminal - Preview Wave 1
Duel Terminal - Preview Wave 2
Duel Terminal 1
Duelist Pack: Battle City
Duelist Pack: Yugi
Duelist Saga
Forbidden Legacy
Ghosts From the Past
Hidden Arsenal: Chapter 1
KC Grand Tournament 2021 prize card
Legendary Collection
Legendary Dragon Decks
Maximum Gold
Shonen Jump Vol. 9, Issue 4 promotional card
Speed Duel Starter Decks: Destiny Masters
Speed Duel Starter Decks: 